In [1]:
import pandas as pd

from pli import generate_pli, pli_intersection_optimized
from candidates import prepare_Fk, generate_candidates

Make a toy dataset to play with

In [2]:
data = {
    'a': ['C', 'A', 'A', 'B', 'C'],
    'b': [3, 1, 1, 1, 1],
    'c': ['X', 'X', 'Y', 'X', 'X'],
    'd': [722, 289, 189, 289, 289],
    'e': [112, 553, 583, 513 ,553]
}

df = pd.DataFrame(data)
df

,a,b,c,d,e
0,C,3,X,722,112
1,A,1,X,289,553
2,A,1,Y,189,583
3,B,1,X,289,513
4,C,1,X,289,553


# Candidates generation

<div>
<img src="img/candidates_generation.png" width="700"/>
</div>

In [3]:
Fk = prepare_Fk(df)
Fk

[('a', {1: [0, 4], 2: [1, 2]}),
 ('b', {2: [1, 2, 3, 4]}),
 ('c', {1: [0, 1, 3, 4]}),
 ('d', {2: [1, 3, 4]}),
 ('e', {2: [1, 4]})]

In [4]:
l1 = generate_candidates(Fk)
l1

[('ab', {(2, 2): [1, 2]}),
 ('ac', {(1, 1): [0, 4]}),
 ('ad', {}),
 ('ae', {}),
 ('bc', {(2, 1): [1, 3, 4]}),
 ('bd', {(2, 2): [1, 3, 4]}),
 ('be', {(2, 2): [1, 4]}),
 ('cd', {(1, 2): [1, 3, 4]}),
 ('ce', {(1, 2): [1, 4]}),
 ('de', {(2, 2): [1, 4]})]

In [5]:
l2 = generate_candidates(l1)
l2

[('abc', {}),
 ('abd', {}),
 ('abe', {}),
 ('acd', {}),
 ('ace', {}),
 ('ade', {}),
 ('bcd', {((2, 1), (2, 2)): [1, 3, 4]}),
 ('bce', {((2, 1), (2, 2)): [1, 4]}),
 ('bde', {((2, 2), (2, 2)): [1, 4]}),
 ('cde', {((1, 2), (1, 2)): [1, 4]})]

# Pruning

<div>
<img src="img/pruning.png" width="700"/>
</div>